# Test of New Orbit Constructors

## Imports

In [106]:
import numpy as np
import pandas as pd

import plotly.graph_objs as go

from ostk.physics.units import Mass
from ostk.physics.units import Angle
from ostk.physics.units import Length
from ostk.physics.time import Scale
from ostk.physics.time import Instant
from ostk.physics.time import Duration
from ostk.physics.time import Interval
from ostk.physics.time import DateTime
from ostk.physics.time import Time
from ostk.physics.coordinate import Position
from ostk.physics.coordinate import Velocity
from ostk.physics.coordinate import Frame
from ostk.physics.coordinate.spherical import LLA
from ostk.physics import Environment
from ostk.physics.environment.objects.celestial_bodies import Earth

from ostk.astrodynamics import Trajectory
from ostk.astrodynamics.trajectory import Orbit
from ostk.astrodynamics.trajectory.orbit.models import Kepler
from ostk.astrodynamics.trajectory.orbit.models.kepler import COE

from ostk.astrodynamics.utilities import convert_state
from ostk.astrodynamics.display import create_2d_map, create_3d_globe

---

## Definitions

**Define the physical `Environment`**

In [107]:
environment = Environment.default()

**Define the type of orbit you want with factory constructors**

In [108]:
epoch = Instant.date_time(DateTime(2018, 1, 1, 0, 0, 0), Scale.UTC)

### Define the geosynchronous orbit's inclination and longitude

In [109]:
inclination = Angle.degrees(30.0)
longitude = Angle.degrees(45.0)

In [110]:
orbit = Orbit.geo_synchronous(
    epoch, inclination, longitude, environment.access_celestial_object_with_name("Earth")
)

### Define a propagation window

In [111]:
propagation_duration = Duration.seconds(86400.0)

interval = Interval.closed(epoch, epoch + propagation_duration)
instants = interval.generate_grid(Duration.minutes(1.0))

## Propagation

**Obtain the `Propagated` `States` for the defined propagation window**

In [112]:
propagated_state_array = orbit.get_states_at(instants)

In [113]:
orbit_data = [
    convert_state(state.get_instant(), state) for state in propagated_state_array
]

In [114]:
orbit_df = pd.DataFrame(
    orbit_data,
    columns=[
        "$Time^{UTC}$",
        "$MJD^{UTC}$",
        "$x_{x}^{ECI}$",
        "$x_{y}^{ECI}$",
        "$x_{z}^{ECI}$",
        "$v_{x}^{ECI}$",
        "$v_{y}^{ECI}$",
        "$v_{z}^{ECI}$",
        "$Latitude$",
        "$Longitude$",
        "$Altitude$",
    ],
)

## Display

2D plot, over **World Map**:

In [115]:
figure = create_2d_map(
    data=go.Scattergeo(
        lon=orbit_df["$Longitude$"],
        lat=orbit_df["$Latitude$"],
        mode="lines",
        line=go.scattergeo.Line(width=1, color="red"),
    ),
)

figure.show()

3D plot, in **Earth Fixed** frame:

In [116]:
figure = create_3d_globe(
    data=[
        go.Scattergeo(
            lon=orbit_df["$Longitude$"],
            lat=orbit_df["$Latitude$"],
            mode="lines",
            line=go.scattergeo.Line(width=2, color="rgb(255, 62, 79)"),
        )
    ],
)

figure.show()